New Notebook Created by Jupyter MCP Server

In [1]:
# 创建训练集和验证集
# ============================================
import pandas as pd
import numpy as np
from glob import glob
import os
import warnings
warnings.filterwarnings('ignore')

# 路径配置
DATA_ROOT = 'D:/010_CodePrograms/L/LLM_su7/小米SU7数据_2'
RESULTS_DIR = 'D:/010_CodePrograms/L/LLM_su7/results'

# 1. 加载去重后的评论数据
print("【加载数据】")
comments_df = pd.read_csv(f'{RESULTS_DIR}/热门微博评论数据_去重.csv', encoding='utf-8-sig')
print(f"评论数据: {len(comments_df):,} 条")
print(f"评论数据列: {list(comments_df.columns)}")

【加载数据】


评论数据: 273,937 条
评论数据列: ['原文链接', '根评论ID', '父评论ID', '评论ID', '用户ID', '父用户昵称', '用户昵称', '评论内容', '发布时间', '子评论数', '点赞数', '用户认证', '用户总评论数', '用户总转发数', '用户总点赞数', '是否是一级评论', 'crawl_date', '内容清洗']


In [2]:
# 2. 加载热门微博数据（获取微博文案）
print("\n【加载热门微博数据】")
weibo_df = pd.read_csv(f'{RESULTS_DIR}/热门微博_去重.csv', encoding='utf-8-sig')
print(f"热门微博: {len(weibo_df):,} 条")
print(f"热门微博列: {list(weibo_df.columns)}")

# 创建 weibo_url -> weibo_content 映射
weibo_content_map = dict(zip(weibo_df['weibo_url'], weibo_df['weibo_content']))
print(f"微博文案映射数: {len(weibo_content_map):,}")


【加载热门微博数据】
热门微博: 4,186 条
热门微博列: ['author_name', 'author_url', 'weibo_url', 'publish_time', 'weibo_content', 'repost_count', 'comment_count', 'like_count', 'crawl_date', '内容清洗']
微博文案映射数: 4,153


In [3]:
# 3. 创建评论ID -> 评论内容映射（用于获取根评论/父评论文案）
print("\n【创建评论文案映射】")

# 标准化评论ID（移除.0后缀）
def normalize_id(id_val):
    if pd.isna(id_val):
        return None
    return str(int(float(id_val))) if not pd.isna(id_val) else None

comments_df['评论ID_str'] = comments_df['评论ID'].apply(normalize_id)
comments_df['根评论ID_str'] = comments_df['根评论ID'].apply(normalize_id)
comments_df['父评论ID_str'] = comments_df['父评论ID'].apply(normalize_id)

# 创建评论ID -> 评论内容映射
comment_content_map = dict(zip(comments_df['评论ID_str'], comments_df['评论内容']))
print(f"评论文案映射数: {len(comment_content_map):,}")


【创建评论文案映射】


评论文案映射数: 273,937


In [4]:
# 4. 构建特征表
print("\n【构建特征表】")

# 添加微博文案
comments_df['微博文案'] = comments_df['原文链接'].map(weibo_content_map)

# 添加根评论文案（一级评论的根评论为空）
comments_df['根评论文案'] = comments_df['根评论ID_str'].map(comment_content_map)

# 添加父评论文案
# 对于一级评论，父评论ID实际上是微博作者ID，不是评论ID
# 需要判断：如果是一级评论，父评论文案应该是微博文案；否则是父评论内容
def get_parent_content(row):
    if row['是否是一级评论']:
        return row['微博文案']  # 一级评论的父内容是微博
    else:
        return comment_content_map.get(row['父评论ID_str'], None)

comments_df['父评论文案'] = comments_df.apply(get_parent_content, axis=1)

# 查看缺失情况
print(f"微博文案缺失: {comments_df['微博文案'].isna().sum():,}")
print(f"根评论文案缺失: {comments_df['根评论文案'].isna().sum():,} (一级评论无根评论)")
print(f"父评论文案缺失: {comments_df['父评论文案'].isna().sum():,}")


【构建特征表】


微博文案缺失: 35
根评论文案缺失: 156,171 (一级评论无根评论)
父评论文案缺失: 1,500


In [5]:
# 5. 选择并重命名特征列
print("\n【选择特征列】")

# 处理用户统计数据（可能有逗号）
def clean_number(x):
    if isinstance(x, str):
        return int(x.replace(',', ''))
    return int(x) if not pd.isna(x) else 0

comments_df['用户总转发数_clean'] = comments_df['用户总转发数'].apply(clean_number)
comments_df['用户总评论数_clean'] = comments_df['用户总评论数'].apply(clean_number)
comments_df['用户总点赞数_clean'] = comments_df['用户总点赞数'].apply(clean_number)

# 选择特征
feature_df = comments_df[[
    # 评论本身文案
    '评论内容',
    # 微博/根评论/父评论文案
    '微博文案',
    '根评论文案',
    '父评论文案',
    # 时间
    '发布时间',
    # 作者统计
    '用户总转发数_clean',
    '用户总评论数_clean', 
    '用户总点赞数_clean',
    # 用户是否认证
    '用户认证',
    # 是否是一级评论
    '是否是一级评论',
    # 评论本身指标（评论没有转发数，只有子评论数和点赞数）
    '子评论数',
    '点赞数'
]].copy()

# 重命名列
feature_df.columns = [
    '评论文案',
    '微博文案',
    '根评论文案',
    '父评论文案',
    '发布时间',
    '用户总转发数',
    '用户总评论数',
    '用户总点赞数',
    '用户是否认证',
    '是否一级评论',
    '子评论数',
    '点赞数'
]

print(f"特征表大小: {len(feature_df):,} 条")
print(f"特征列: {list(feature_df.columns)}")
print(f"\n特征数据示例:")
feature_df.head(3)


【选择特征列】


特征表大小: 273,937 条
特征列: ['评论文案', '微博文案', '根评论文案', '父评论文案', '发布时间', '用户总转发数', '用户总评论数', '用户总点赞数', '用户是否认证', '是否一级评论', '子评论数', '点赞数']

特征数据示例:


,评论文案,微博文案,根评论文案,父评论文案,发布时间,用户总转发数,用户总评论数,用户总点赞数,用户是否认证,是否一级评论,子评论数,点赞数
0,非常好非常好，好产品应当像小米汽车一样会自动宣传[鼓掌][鼓掌][鼓掌][打call][打c...,众所周知，小米汽车的广告都是车主拍的👏 一起来欣赏大片，小米SU7 Ultra车主出品。 L...,NaN,众所周知，小米汽车的广告都是车主拍的👏 一起来欣赏大片，小米SU7 Ultra车主出品。 L...,2025-03-27 18:26:25,610,25928,40859,True,True,3,127.0
1,对对对啊,众所周知，小米汽车的广告都是车主拍的👏 一起来欣赏大片，小米SU7 Ultra车主出品。 L...,非常好非常好，好产品应当像小米汽车一样会自动宣传[鼓掌][鼓掌][鼓掌][打call][打c...,非常好非常好，好产品应当像小米汽车一样会自动宣传[鼓掌][鼓掌][鼓掌][打call][打c...,2025-03-28 07:21:20,19,28,35,False,False,0,0.0
2,加油加油加油,众所周知，小米汽车的广告都是车主拍的👏 一起来欣赏大片，小米SU7 Ultra车主出品。 L...,非常好非常好，好产品应当像小米汽车一样会自动宣传[鼓掌][鼓掌][鼓掌][打call][打c...,NaN,2025-03-27 18:28:55,610,25928,40859,True,False,0,0.0


In [6]:
# 6. 划分训练集和验证集
# 规则：评论文案重复的数据中，时间上第一次出现的必须放在训练集
print("\n【划分训练集和验证集】")

# 转换时间列
feature_df['发布时间'] = pd.to_datetime(feature_df['发布时间'])

# 按评论文案和时间排序
feature_df = feature_df.sort_values(['评论文案', '发布时间']).reset_index(drop=True)

# 标记每个评论文案组内的顺序
feature_df['文案顺序'] = feature_df.groupby('评论文案').cumcount()

# 第一次出现的（顺序=0）必须放入训练集
first_occurrence = feature_df[feature_df['文案顺序'] == 0].copy()
later_occurrence = feature_df[feature_df['文案顺序'] > 0].copy()

print(f"首次出现的评论: {len(first_occurrence):,}")
print(f"重复出现的评论: {len(later_occurrence):,}")

# 计算需要的验证集大小（总量的10%）
total_size = len(feature_df)
val_size = int(total_size * 0.1)
print(f"\n目标验证集大小: {val_size:,} (总量的10%)")


【划分训练集和验证集】


首次出现的评论: 246,090
重复出现的评论: 25,362

目标验证集大小: 27,393 (总量的10%)


In [7]:
# 7. 分配数据到训练集和验证集
import numpy as np
np.random.seed(42)

# 策略：
# 1. 首次出现的评论必须放训练集
# 2. 从重复出现的评论中抽取一部分放验证集
# 3. 如果重复评论不够，从首次出现中抽取（但要保证该文案的其他重复也都在训练集）

# 重复评论数量
later_count = len(later_occurrence)

if later_count >= val_size:
    # 重复评论足够，直接从中抽取
    val_indices = np.random.choice(later_occurrence.index, size=val_size, replace=False)
    val_df = later_occurrence.loc[val_indices]
    train_df = pd.concat([first_occurrence, later_occurrence.drop(val_indices)])
else:
    # 重复评论不够，全部放验证集，再从首次出现中补充
    # 但要确保：如果某个文案的首次出现放入验证集，该文案不能有重复出现
    need_more = val_size - later_count
    
    # 找出没有重复的首次出现评论（可以安全放入验证集）
    unique_contents = first_occurrence[~first_occurrence['评论文案'].isin(later_occurrence['评论文案'])]
    print(f"无重复的首次评论数: {len(unique_contents):,}")
    
    if len(unique_contents) >= need_more:
        # 从无重复评论中抽取
        extra_val_indices = np.random.choice(unique_contents.index, size=need_more, replace=False)
        val_df = pd.concat([later_occurrence, unique_contents.loc[extra_val_indices]])
        train_df = first_occurrence.drop(extra_val_indices)
    else:
        # 无重复评论也不够，全部使用
        val_df = pd.concat([later_occurrence, unique_contents])
        train_df = first_occurrence.drop(unique_contents.index)
        print(f"警告: 验证集大小 {len(val_df)} 小于目标 {val_size}")

print(f"\n训练集大小: {len(train_df):,}")
print(f"验证集大小: {len(val_df):,}")
print(f"比例: {len(train_df)/len(feature_df)*100:.1f}% : {len(val_df)/len(feature_df)*100:.1f}%")

无重复的首次评论数: 240,467

训练集大小: 244,059
验证集大小: 27,393
比例: 89.1% : 10.0%


In [8]:
# 8. 验证划分的正确性
print("\n【验证划分正确性】")

# 检查：重复评论的首次出现是否都在训练集中
duplicated_contents = later_occurrence['评论文案'].unique()
first_of_duplicated = first_occurrence[first_occurrence['评论文案'].isin(duplicated_contents)]

in_train = first_of_duplicated.index.isin(train_df.index)
print(f"有重复的首次评论数: {len(first_of_duplicated):,}")
print(f"其中在训练集的: {in_train.sum():,}")
print(f"其中在验证集的: {(~in_train).sum():,}")

if (~in_train).sum() == 0:
    print("✓ 验证通过：所有重复评论的首次出现都在训练集中")


【验证划分正确性】
有重复的首次评论数: 5,623
其中在训练集的: 5,623
其中在验证集的: 0
✓ 验证通过：所有重复评论的首次出现都在训练集中


In [9]:
# 9. 保存训练集和验证集
print("\n【保存数据】")

# 删除辅助列
train_df = train_df.drop(columns=['文案顺序'])
val_df = val_df.drop(columns=['文案顺序'])

# 重置索引
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

# 保存
train_path = f'{RESULTS_DIR}/train.csv'
val_path = f'{RESULTS_DIR}/val.csv'

train_df.to_csv(train_path, index=False, encoding='utf-8-sig')
val_df.to_csv(val_path, index=False, encoding='utf-8-sig')

print(f"训练集已保存: {train_path}")
print(f"验证集已保存: {val_path}")

print(f"\n【最终数据统计】")
print(f"训练集: {len(train_df):,} 条 ({len(train_df)/len(feature_df)*100:.1f}%)")
print(f"验证集: {len(val_df):,} 条 ({len(val_df)/len(feature_df)*100:.1f}%)")
print(f"\n特征列: {list(train_df.columns)}")


【保存数据】


训练集已保存: D:/010_CodePrograms/L/LLM_su7/results/train.csv
验证集已保存: D:/010_CodePrograms/L/LLM_su7/results/val.csv

【最终数据统计】
训练集: 244,059 条 (89.1%)
验证集: 27,393 条 (10.0%)

特征列: ['评论文案', '微博文案', '根评论文案', '父评论文案', '发布时间', '用户总转发数', '用户总评论数', '用户总点赞数', '用户是否认证', '是否一级评论', '子评论数', '点赞数']


In [10]:
# 10. 数据概览
print("=" * 60)
print("【数据集概览】")
print("=" * 60)

print("\n--- 训练集 ---")
print(train_df.info())
print(f"\n缺失值统计:")
print(train_df.isnull().sum())

print("\n\n--- 验证集 ---")
print(f"缺失值统计:")
print(val_df.isnull().sum())

print("\n\n--- 数值特征分布 ---")
print("训练集:")
print(train_df[['用户总转发数', '用户总评论数', '用户总点赞数', '子评论数', '点赞数']].describe())

【数据集概览】

--- 训练集 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244059 entries, 0 to 244058
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   评论文案    244059 non-null  object        
 1   微博文案    244026 non-null  object        
 2   根评论文案   108322 non-null  object        
 3   父评论文案   242853 non-null  object        
 4   发布时间    244059 non-null  datetime64[ns]
 5   用户总转发数  244059 non-null  int64         
 6   用户总评论数  244059 non-null  int64         
 7   用户总点赞数  244059 non-null  int64         
 8   用户是否认证  244059 non-null  bool          
 9   是否一级评论  244059 non-null  bool          
 10  子评论数    244059 non-null  int64         
 11  点赞数     180854 non-null  float64       
dtypes: bool(2), datetime64[ns](1), float64(1), int64(4), object(4)
memory usage: 19.1+ MB
None

缺失值统计:
评论文案           0
微博文案          33
根评论文案     135737
父评论文案       1206
发布时间           0
用户总转发数         0
用户总评论数         0
用户总点赞数         

In [11]:
print("=" * 60)
print("【完成】")
print("=" * 60)
print(f"""
已创建训练集和验证集:
• results/train.csv: {len(train_df):,} 条 (89.1%)
• results/val.csv: {len(val_df):,} 条 (10.0%)

特征说明:
1. 评论文案: 评论本身的文案内容
2. 微博文案: 该评论所属微博的内容
3. 根评论文案: 根评论内容（一级评论为空）
4. 父评论文案: 父评论/微博内容
5. 发布时间: 评论发布时间
6. 用户总转发数: 评论作者的历史转发总数
7. 用户总评论数: 评论作者的历史评论总数
8. 用户总点赞数: 评论作者的历史点赞总数
9. 用户是否认证: 评论作者是否认证用户
10. 是否一级评论: 是否为一级评论（直接评论微博）
11. 子评论数: 该评论获得的回复数
12. 点赞数: 该评论获得的点赞数

划分规则:
• 评论文案重复的数据中，时间上首次出现的必须在训练集
• 训练集:验证集 ≈ 9:1
""")

【完成】

已创建训练集和验证集:
• results/train.csv: 244,059 条 (89.1%)
• results/val.csv: 27,393 条 (10.0%)

特征说明:
1. 评论文案: 评论本身的文案内容
2. 微博文案: 该评论所属微博的内容
3. 根评论文案: 根评论内容（一级评论为空）
4. 父评论文案: 父评论/微博内容
5. 发布时间: 评论发布时间
6. 用户总转发数: 评论作者的历史转发总数
7. 用户总评论数: 评论作者的历史评论总数
8. 用户总点赞数: 评论作者的历史点赞总数
9. 用户是否认证: 评论作者是否认证用户
10. 是否一级评论: 是否为一级评论（直接评论微博）
11. 子评论数: 该评论获得的回复数
12. 点赞数: 该评论获得的点赞数

划分规则:
• 评论文案重复的数据中，时间上首次出现的必须在训练集
• 训练集:验证集 ≈ 9:1



In [12]:
# 11. 重新划分：训练集/验证集/测试集 = 8:1:1
print("=" * 60)
print("【重新划分：8:1:1】")
print("=" * 60)

np.random.seed(42)

# 重新加载特征数据
feature_df = feature_df.sort_values(['评论文案', '发布时间']).reset_index(drop=True)
feature_df['文案顺序'] = feature_df.groupby('评论文案').cumcount()

first_occurrence = feature_df[feature_df['文案顺序'] == 0].copy()
later_occurrence = feature_df[feature_df['文案顺序'] > 0].copy()

total_size = len(feature_df)
val_size = int(total_size * 0.1)  # 10%
test_size = int(total_size * 0.1)  # 10%
val_test_size = val_size + test_size  # 20%

print(f"总数据量: {total_size:,}")
print(f"目标训练集: {total_size - val_test_size:,} (80%)")
print(f"目标验证集: {val_size:,} (10%)")
print(f"目标测试集: {test_size:,} (10%)")
print(f"\n首次出现的评论: {len(first_occurrence):,}")
print(f"重复出现的评论: {len(later_occurrence):,}")

【重新划分：8:1:1】


总数据量: 273,937
目标训练集: 219,151 (80%)
目标验证集: 27,393 (10%)
目标测试集: 27,393 (10%)

首次出现的评论: 246,090
重复出现的评论: 25,362


In [13]:
# 12. 分配数据
# 规则：重复评论的首次出现必须在训练集

later_count = len(later_occurrence)

# 找出无重复的首次出现评论
unique_contents = first_occurrence[~first_occurrence['评论文案'].isin(later_occurrence['评论文案'])]
print(f"无重复的首次评论数: {len(unique_contents):,}")

# 从重复评论和无重复首次评论中抽取验证集和测试集
available_for_val_test = pd.concat([later_occurrence, unique_contents])
print(f"可用于验证/测试的数据: {len(available_for_val_test):,}")

# 随机打乱并分配
available_shuffled = available_for_val_test.sample(frac=1, random_state=42).reset_index(drop=True)

val_df = available_shuffled.iloc[:val_size].copy()
test_df = available_shuffled.iloc[val_size:val_size+test_size].copy()
used_indices = set(val_df.index.tolist() + test_df.index.tolist())

# 剩余的放入训练集
train_from_available = available_shuffled.iloc[val_size+test_size:].copy()

# 加上有重复的首次评论（必须在训练集）
first_with_dup = first_occurrence[first_occurrence['评论文案'].isin(later_occurrence['评论文案'])]
train_df = pd.concat([first_with_dup, train_from_available])

print(f"\n训练集: {len(train_df):,} ({len(train_df)/total_size*100:.1f}%)")
print(f"验证集: {len(val_df):,} ({len(val_df)/total_size*100:.1f}%)")
print(f"测试集: {len(test_df):,} ({len(test_df)/total_size*100:.1f}%)")

无重复的首次评论数: 240,467
可用于验证/测试的数据: 265,829

训练集: 216,666 (79.1%)
验证集: 27,393 (10.0%)
测试集: 27,393 (10.0%)


In [14]:
# 13. 验证划分正确性
print("\n【验证划分正确性】")

# 检查：重复评论的首次出现是否都在训练集中
duplicated_contents = later_occurrence['评论文案'].unique()
first_of_duplicated = first_occurrence[first_occurrence['评论文案'].isin(duplicated_contents)]

# 获取原始索引
train_original_idx = set(train_df.index if '文案顺序' in train_df.columns else [])

in_train = first_of_duplicated.index.isin(train_df.index)
print(f"有重复的首次评论数: {len(first_of_duplicated):,}")
print(f"其中在训练集的: {in_train.sum():,}")

if in_train.sum() == len(first_of_duplicated):
    print("✓ 验证通过：所有重复评论的首次出现都在训练集中")
else:
    print("✗ 验证失败")

# 检查数据无交叉
train_comments = set(train_df['评论文案'].tolist())
val_comments = set(val_df['评论文案'].tolist())
test_comments = set(test_df['评论文案'].tolist())

print(f"\n数据总量验证: {len(train_df) + len(val_df) + len(test_df):,} = {total_size:,}")


【验证划分正确性】
有重复的首次评论数: 5,623
其中在训练集的: 5,623
✓ 验证通过：所有重复评论的首次出现都在训练集中

数据总量验证: 271,452 = 273,937


In [15]:
# 14. 修正划分逻辑
print("【修正划分逻辑】")

np.random.seed(42)

# 重新开始
feature_df_clean = feature_df.copy()
feature_df_clean = feature_df_clean.sort_values(['评论文案', '发布时间']).reset_index(drop=True)
feature_df_clean['文案顺序'] = feature_df_clean.groupby('评论文案').cumcount()

# 分离首次出现和重复出现
first_occ = feature_df_clean[feature_df_clean['文案顺序'] == 0].copy()
later_occ = feature_df_clean[feature_df_clean['文案顺序'] > 0].copy()

# 有重复的首次评论（必须在训练集）
has_dup_mask = first_occ['评论文案'].isin(later_occ['评论文案'])
first_with_dup = first_occ[has_dup_mask].copy()  # 必须在训练集
first_no_dup = first_occ[~has_dup_mask].copy()   # 可以分配到任意集合

print(f"首次出现且有重复（必须在训练集）: {len(first_with_dup):,}")
print(f"首次出现且无重复（可分配）: {len(first_no_dup):,}")
print(f"重复出现（可分配）: {len(later_occ):,}")

# 可分配的数据
available = pd.concat([first_no_dup, later_occ]).sample(frac=1, random_state=42)
print(f"可分配总数: {len(available):,}")

# 计算各集合大小
total = len(feature_df_clean)
val_size = int(total * 0.1)
test_size = int(total * 0.1)

# 分配
val_df = available.iloc[:val_size].copy()
test_df = available.iloc[val_size:val_size+test_size].copy()
train_from_available = available.iloc[val_size+test_size:].copy()
train_df = pd.concat([first_with_dup, train_from_available])

print(f"\n最终划分:")
print(f"训练集: {len(train_df):,} ({len(train_df)/total*100:.1f}%)")
print(f"验证集: {len(val_df):,} ({len(val_df)/total*100:.1f}%)")
print(f"测试集: {len(test_df):,} ({len(test_df)/total*100:.1f}%)")
print(f"总计: {len(train_df)+len(val_df)+len(test_df):,} = {total:,}")

【修正划分逻辑】


首次出现且有重复（必须在训练集）: 5,623
首次出现且无重复（可分配）: 240,467
重复出现（可分配）: 25,362
可分配总数: 265,829

最终划分:
训练集: 216,666 (79.1%)
验证集: 27,393 (10.0%)
测试集: 27,393 (10.0%)
总计: 271,452 = 273,937


In [16]:
# 15. 检查数据缺失
print("【检查数据差异】")
print(f"feature_df_clean 总数: {len(feature_df_clean):,}")
print(f"first_occ + later_occ: {len(first_occ) + len(later_occ):,}")

# 检查是否有空评论文案
null_comments = feature_df_clean['评论文案'].isna().sum()
empty_comments = (feature_df_clean['评论文案'] == '').sum()
print(f"\n评论文案为空(NaN): {null_comments:,}")
print(f"评论文案为空字符串: {empty_comments:,}")

# 检查空评论的情况
if null_comments > 0 or empty_comments > 0:
    empty_mask = feature_df_clean['评论文案'].isna() | (feature_df_clean['评论文案'] == '')
    print(f"\n空评论数据量: {empty_mask.sum():,}")
    print("这些空评论会被groupby合并，导致计数差异")

【检查数据差异】
feature_df_clean 总数: 273,937
first_occ + later_occ: 271,452

评论文案为空(NaN): 2,485
评论文案为空字符串: 0

空评论数据量: 2,485
这些空评论会被groupby合并，导致计数差异


In [17]:
# 16. 处理空评论并重新划分
print("【处理空评论并重新划分】")

np.random.seed(42)

# 分离空评论和非空评论
empty_mask = feature_df_clean['评论文案'].isna()
df_empty = feature_df_clean[empty_mask].copy()
df_non_empty = feature_df_clean[~empty_mask].copy()

print(f"空评论: {len(df_empty):,}")
print(f"非空评论: {len(df_non_empty):,}")

# 对非空评论按文案分组
df_non_empty = df_non_empty.sort_values(['评论文案', '发布时间']).reset_index(drop=True)
df_non_empty['文案顺序'] = df_non_empty.groupby('评论文案').cumcount()

first_occ = df_non_empty[df_non_empty['文案顺序'] == 0].copy()
later_occ = df_non_empty[df_non_empty['文案顺序'] > 0].copy()

# 有重复的首次评论必须在训练集
has_dup_mask = first_occ['评论文案'].isin(later_occ['评论文案'])
first_with_dup = first_occ[has_dup_mask].copy()
first_no_dup = first_occ[~has_dup_mask].copy()

# 可分配的数据（包括空评论）
available = pd.concat([first_no_dup, later_occ, df_empty]).sample(frac=1, random_state=42)

# 计算各集合大小
total = len(feature_df_clean)
val_size = int(total * 0.1)
test_size = int(total * 0.1)

# 分配
val_df = available.iloc[:val_size].copy()
test_df = available.iloc[val_size:val_size+test_size].copy()
train_from_available = available.iloc[val_size+test_size:].copy()
train_df = pd.concat([first_with_dup, train_from_available])

print(f"\n最终划分:")
print(f"训练集: {len(train_df):,} ({len(train_df)/total*100:.1f}%)")
print(f"验证集: {len(val_df):,} ({len(val_df)/total*100:.1f}%)")
print(f"测试集: {len(test_df):,} ({len(test_df)/total*100:.1f}%)")
print(f"总计: {len(train_df)+len(val_df)+len(test_df):,} = {total:,} ✓")

【处理空评论并重新划分】
空评论: 2,485
非空评论: 271,452



最终划分:
训练集: 219,151 (80.0%)
验证集: 27,393 (10.0%)
测试集: 27,393 (10.0%)
总计: 273,937 = 273,937 ✓


In [18]:
# 17. 保存到根目录
print("【保存数据到根目录】")

ROOT_DIR = 'D:/010_CodePrograms/L/LLM_su7'

# 删除辅助列
for df in [train_df, val_df, test_df]:
    if '文案顺序' in df.columns:
        df.drop(columns=['文案顺序'], inplace=True)

# 重置索引
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# 保存
train_df.to_csv(f'{ROOT_DIR}/train.csv', index=False, encoding='utf-8-sig')
val_df.to_csv(f'{ROOT_DIR}/val.csv', index=False, encoding='utf-8-sig')
test_df.to_csv(f'{ROOT_DIR}/test.csv', index=False, encoding='utf-8-sig')

print(f"✓ train.csv: {len(train_df):,} 条 (80.0%)")
print(f"✓ val.csv: {len(val_df):,} 条 (10.0%)")
print(f"✓ test.csv: {len(test_df):,} 条 (10.0%)")

# 删除results目录下的旧文件
import os
old_files = [f'{RESULTS_DIR}/train.csv', f'{RESULTS_DIR}/val.csv']
for f in old_files:
    if os.path.exists(f):
        os.remove(f)
        print(f"已删除旧文件: {f}")

【保存数据到根目录】


✓ train.csv: 219,151 条 (80.0%)
✓ val.csv: 27,393 条 (10.0%)
✓ test.csv: 27,393 条 (10.0%)
已删除旧文件: D:/010_CodePrograms/L/LLM_su7/results/train.csv
已删除旧文件: D:/010_CodePrograms/L/LLM_su7/results/val.csv


In [19]:
print("=" * 60)
print("【完成】")
print("=" * 60)
print(f"""
已创建训练集/验证集/测试集 (8:1:1):
• train.csv: {len(train_df):,} 条 (80.0%)
• val.csv: {len(val_df):,} 条 (10.0%)
• test.csv: {len(test_df):,} 条 (10.0%)

保存位置: D:/010_CodePrograms/L/LLM_su7/

特征列 (12列):
  评论文案, 微博文案, 根评论文案, 父评论文案, 发布时间,
  用户总转发数, 用户总评论数, 用户总点赞数,
  用户是否认证, 是否一级评论, 子评论数, 点赞数

划分规则:
• 评论文案重复时，时间上首次出现的必须在训练集 ✓
• 空评论(2,485条)随机分配到各集合
""")

【完成】

已创建训练集/验证集/测试集 (8:1:1):
• train.csv: 219,151 条 (80.0%)
• val.csv: 27,393 条 (10.0%)
• test.csv: 27,393 条 (10.0%)

保存位置: D:/010_CodePrograms/L/LLM_su7/

特征列 (12列):
  评论文案, 微博文案, 根评论文案, 父评论文案, 发布时间,
  用户总转发数, 用户总评论数, 用户总点赞数,
  用户是否认证, 是否一级评论, 子评论数, 点赞数

划分规则:
• 评论文案重复时，时间上首次出现的必须在训练集 ✓
• 空评论(2,485条)随机分配到各集合



In [20]:
# 19. 重新处理数据：去除空评论
print("=" * 60)
print("【重新处理数据】")
print("=" * 60)

# 重新加载评论数据
comments_df = pd.read_csv(f'{RESULTS_DIR}/热门微博评论数据_去重.csv', encoding='utf-8-sig')
print(f"原始评论数据: {len(comments_df):,} 条")

# 去除空评论
empty_mask = comments_df['评论内容'].isna() | (comments_df['评论内容'] == '')
print(f"空评论数: {empty_mask.sum():,}")

comments_df = comments_df[~empty_mask].reset_index(drop=True)
print(f"去除空评论后: {len(comments_df):,} 条")

# 重新保存
comments_df.to_csv(f'{RESULTS_DIR}/热门微博评论数据_去重.csv', index=False, encoding='utf-8-sig')
print(f"已更新保存: {RESULTS_DIR}/热门微博评论数据_去重.csv")

【重新处理数据】


原始评论数据: 273,937 条
空评论数: 2,485
去除空评论后: 271,452 条


已更新保存: D:/010_CodePrograms/L/LLM_su7/results/热门微博评论数据_去重.csv


In [21]:
# 19. 去除空评论并重新保存
print("=" * 60)
print("【去除空评论】")
print("=" * 60)

# 重新加载原始去重数据
comments_raw = pd.read_csv(f'{RESULTS_DIR}/热门微博评论数据_去重.csv', encoding='utf-8-sig')
print(f"原始数据: {len(comments_raw):,} 条")

# 检查空评论
null_count = comments_raw['评论内容'].isna().sum()
empty_count = (comments_raw['评论内容'] == '').sum()
print(f"评论内容为NaN: {null_count:,}")
print(f"评论内容为空字符串: {empty_count:,}")

# 去除空评论
comments_clean = comments_raw[comments_raw['评论内容'].notna() & (comments_raw['评论内容'] != '')].copy()
print(f"\n去除空评论后: {len(comments_clean):,} 条")

# 重新保存
comments_clean.to_csv(f'{RESULTS_DIR}/热门微博评论数据_去重.csv', index=False, encoding='utf-8-sig')
print(f"已保存: {RESULTS_DIR}/热门微博评论数据_去重.csv")

【去除空评论】


原始数据: 271,452 条
评论内容为NaN: 0
评论内容为空字符串: 0

去除空评论后: 271,452 条


已保存: D:/010_CodePrograms/L/LLM_su7/results/热门微博评论数据_去重.csv


In [22]:
# 20. 重新加载并构建特征（包含微博转发数/评论数/点赞数）
print("=" * 60)
print("【重新构建特征表】")
print("=" * 60)

# 加载评论数据
comments_df = pd.read_csv(f'{RESULTS_DIR}/热门微博评论数据_去重.csv', encoding='utf-8-sig')
print(f"评论数据: {len(comments_df):,} 条")

# 加载微博数据（获取转发数/评论数/点赞数）
weibo_df = pd.read_csv(f'{RESULTS_DIR}/热门微博_去重.csv', encoding='utf-8-sig')
print(f"微博数据: {len(weibo_df):,} 条")

# 创建微博特征映射
weibo_features = weibo_df.set_index('weibo_url')[['weibo_content', 'repost_count', 'comment_count', 'like_count']]
weibo_features.columns = ['微博文案', '微博转发数', '微博评论数', '微博点赞数']

print(f"\n微博特征映射数: {len(weibo_features):,}")
print(weibo_features.head(2))

【重新构建特征表】


评论数据: 271,452 条
微博数据: 4,186 条

微博特征映射数: 4,186
                                                                                     微博文案  \
weibo_url                                                                                   
https://weibo.com/7871239944/PkvsRdOvf  众所周知，小米汽车的广告都是车主拍的👏 一起来欣赏大片，小米SU7 Ultra车主出品。 L...   
https://weibo.com/7747056179/Pkpf5e7ul                      小米su7是他的命吧，有同款队友吗 LYTB亮眼的微博视频   

                                        微博转发数  微博评论数   微博点赞数  
weibo_url                                                     
https://weibo.com/7871239944/PkvsRdOvf     48    105  2017.0  
https://weibo.com/7747056179/Pkpf5e7ul     30    280  1619.0  


In [23]:
# 21. 构建完整特征表
print("【构建完整特征表】")

# 标准化评论ID
def normalize_id(id_val):
    if pd.isna(id_val):
        return None
    return str(int(float(id_val)))

comments_df['评论ID_str'] = comments_df['评论ID'].apply(normalize_id)
comments_df['根评论ID_str'] = comments_df['根评论ID'].apply(normalize_id)
comments_df['父评论ID_str'] = comments_df['父评论ID'].apply(normalize_id)

# 创建评论ID -> 评论内容映射
comment_content_map = dict(zip(comments_df['评论ID_str'], comments_df['评论内容']))

# 关联微博特征
comments_df = comments_df.merge(
    weibo_features.reset_index(), 
    left_on='原文链接', 
    right_on='weibo_url', 
    how='left'
).drop(columns=['weibo_url'])

# 添加根评论文案
comments_df['根评论文案'] = comments_df['根评论ID_str'].map(comment_content_map)

# 添加父评论文案
def get_parent_content(row):
    if row['是否是一级评论']:
        return row['微博文案']
    else:
        return comment_content_map.get(row['父评论ID_str'], None)

comments_df['父评论文案'] = comments_df.apply(get_parent_content, axis=1)

print(f"特征构建完成，数据量: {len(comments_df):,}")

【构建完整特征表】


特征构建完成，数据量: 284,109


In [24]:
# 22. 检查merge问题
print("【检查数据膨胀问题】")
print(f"原评论数: 271,452")
print(f"merge后: {len(comments_df):,}")

# 检查微博URL是否有重复
dup_urls = weibo_df['weibo_url'].duplicated().sum()
print(f"\n微博URL重复数: {dup_urls}")

# 重新处理：先去重微博数据
weibo_dedup = weibo_df.drop_duplicates(subset=['weibo_url'], keep='first')
print(f"去重后微博数: {len(weibo_dedup):,}")

# 重新加载评论数据并merge
comments_df = pd.read_csv(f'{RESULTS_DIR}/热门微博评论数据_去重.csv', encoding='utf-8-sig')

weibo_features = weibo_dedup.set_index('weibo_url')[['weibo_content', 'repost_count', 'comment_count', 'like_count']]
weibo_features.columns = ['微博文案', '微博转发数', '微博评论数', '微博点赞数']

comments_df = comments_df.merge(
    weibo_features.reset_index(), 
    left_on='原文链接', 
    right_on='weibo_url', 
    how='left'
).drop(columns=['weibo_url'])

print(f"\n重新merge后数据量: {len(comments_df):,}")

【检查数据膨胀问题】
原评论数: 271,452
merge后: 284,109

微博URL重复数: 33
去重后微博数: 4,153



重新merge后数据量: 271,452


In [25]:
# 23. 继续构建特征
print("【继续构建特征】")

# 标准化评论ID
comments_df['评论ID_str'] = comments_df['评论ID'].apply(normalize_id)
comments_df['根评论ID_str'] = comments_df['根评论ID'].apply(normalize_id)
comments_df['父评论ID_str'] = comments_df['父评论ID'].apply(normalize_id)

# 创建评论ID -> 评论内容映射
comment_content_map = dict(zip(comments_df['评论ID_str'], comments_df['评论内容']))

# 添加根评论文案
comments_df['根评论文案'] = comments_df['根评论ID_str'].map(comment_content_map)

# 添加父评论文案
comments_df['父评论文案'] = comments_df.apply(get_parent_content, axis=1)

# 处理用户统计数据
comments_df['用户总转发数_clean'] = comments_df['用户总转发数'].apply(clean_number)
comments_df['用户总评论数_clean'] = comments_df['用户总评论数'].apply(clean_number)
comments_df['用户总点赞数_clean'] = comments_df['用户总点赞数'].apply(clean_number)

print(f"特征构建完成")
print(f"\n缺失值统计:")
print(f"  微博文案缺失: {comments_df['微博文案'].isna().sum():,}")
print(f"  微博转发数缺失: {comments_df['微博转发数'].isna().sum():,}")
print(f"  根评论文案缺失: {comments_df['根评论文案'].isna().sum():,} (一级评论无根评论)")
print(f"  父评论文案缺失: {comments_df['父评论文案'].isna().sum():,}")

【继续构建特征】


特征构建完成

缺失值统计:
  微博文案缺失: 34
  微博转发数缺失: 0
  根评论文案缺失: 156,171 (一级评论无根评论)
  父评论文案缺失: 1,316


In [26]:
# 24. 选择特征列
print("【选择特征列】")

feature_df = comments_df[[
    # 评论本身文案
    '评论内容',
    # 微博/根评论/父评论文案
    '微博文案',
    '根评论文案',
    '父评论文案',
    # 时间
    '发布时间',
    # 作者统计
    '用户总转发数_clean',
    '用户总评论数_clean', 
    '用户总点赞数_clean',
    # 用户是否认证
    '用户认证',
    # 是否是一级评论
    '是否是一级评论',
    # 评论本身指标
    '子评论数',
    '点赞数',
    # 微博指标（新增）
    '微博转发数',
    '微博评论数',
    '微博点赞数'
]].copy()

# 重命名列
feature_df.columns = [
    '评论文案', '微博文案', '根评论文案', '父评论文案', '发布时间',
    '用户总转发数', '用户总评论数', '用户总点赞数',
    '用户是否认证', '是否一级评论', '子评论数', '点赞数',
    '微博转发数', '微博评论数', '微博点赞数'
]

print(f"特征表大小: {len(feature_df):,} 条")
print(f"特征列 ({len(feature_df.columns)}列): {list(feature_df.columns)}")

【选择特征列】
特征表大小: 271,452 条
特征列 (15列): ['评论文案', '微博文案', '根评论文案', '父评论文案', '发布时间', '用户总转发数', '用户总评论数', '用户总点赞数', '用户是否认证', '是否一级评论', '子评论数', '点赞数', '微博转发数', '微博评论数', '微博点赞数']


In [27]:
# 25. 划分训练集/验证集/测试集 (8:1:1)
print("=" * 60)
print("【划分数据集 8:1:1】")
print("=" * 60)

np.random.seed(42)

# 转换时间列
feature_df['发布时间'] = pd.to_datetime(feature_df['发布时间'])

# 按评论文案和时间排序
feature_df = feature_df.sort_values(['评论文案', '发布时间']).reset_index(drop=True)
feature_df['文案顺序'] = feature_df.groupby('评论文案').cumcount()

# 分离首次出现和重复出现
first_occ = feature_df[feature_df['文案顺序'] == 0].copy()
later_occ = feature_df[feature_df['文案顺序'] > 0].copy()

# 有重复的首次评论（必须在训练集）
has_dup_mask = first_occ['评论文案'].isin(later_occ['评论文案'])
first_with_dup = first_occ[has_dup_mask].copy()
first_no_dup = first_occ[~has_dup_mask].copy()

print(f"首次出现且有重复（必须在训练集）: {len(first_with_dup):,}")
print(f"首次出现且无重复（可分配）: {len(first_no_dup):,}")
print(f"重复出现（可分配）: {len(later_occ):,}")

# 可分配的数据
available = pd.concat([first_no_dup, later_occ]).sample(frac=1, random_state=42)

# 计算各集合大小
total = len(feature_df)
val_size = int(total * 0.1)
test_size = int(total * 0.1)

# 分配
val_df = available.iloc[:val_size].copy()
test_df = available.iloc[val_size:val_size+test_size].copy()
train_from_available = available.iloc[val_size+test_size:].copy()
train_df = pd.concat([first_with_dup, train_from_available])

print(f"\n划分结果:")
print(f"训练集: {len(train_df):,} ({len(train_df)/total*100:.1f}%)")
print(f"验证集: {len(val_df):,} ({len(val_df)/total*100:.1f}%)")
print(f"测试集: {len(test_df):,} ({len(test_df)/total*100:.1f}%)")
print(f"总计: {len(train_df)+len(val_df)+len(test_df):,} = {total:,} ✓")

【划分数据集 8:1:1】


首次出现且有重复（必须在训练集）: 5,623
首次出现且无重复（可分配）: 240,467
重复出现（可分配）: 25,362

划分结果:
训练集: 217,162 (80.0%)
验证集: 27,145 (10.0%)
测试集: 27,145 (10.0%)
总计: 271,452 = 271,452 ✓


In [28]:
# 26. 保存为CSV和PKL两种格式
print("=" * 60)
print("【保存数据】")
print("=" * 60)

ROOT_DIR = 'D:/010_CodePrograms/L/LLM_su7'

# 删除辅助列并重置索引
for df in [train_df, val_df, test_df]:
    if '文案顺序' in df.columns:
        df.drop(columns=['文案顺序'], inplace=True)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# 保存CSV格式
train_df.to_csv(f'{ROOT_DIR}/train.csv', index=False, encoding='utf-8-sig')
val_df.to_csv(f'{ROOT_DIR}/val.csv', index=False, encoding='utf-8-sig')
test_df.to_csv(f'{ROOT_DIR}/test.csv', index=False, encoding='utf-8-sig')

print("CSV格式:")
print(f"  ✓ train.csv: {len(train_df):,} 条")
print(f"  ✓ val.csv: {len(val_df):,} 条")
print(f"  ✓ test.csv: {len(test_df):,} 条")

# 保存PKL格式
train_df.to_pickle(f'{ROOT_DIR}/train.pkl')
val_df.to_pickle(f'{ROOT_DIR}/val.pkl')
test_df.to_pickle(f'{ROOT_DIR}/test.pkl')

print("\nPKL格式:")
print(f"  ✓ train.pkl: {len(train_df):,} 条")
print(f"  ✓ val.pkl: {len(val_df):,} 条")
print(f"  ✓ test.pkl: {len(test_df):,} 条")

# 比较文件大小
import os
for name in ['train', 'val', 'test']:
    csv_size = os.path.getsize(f'{ROOT_DIR}/{name}.csv') / 1024 / 1024
    pkl_size = os.path.getsize(f'{ROOT_DIR}/{name}.pkl') / 1024 / 1024
    print(f"\n{name}: CSV={csv_size:.1f}MB, PKL={pkl_size:.1f}MB (节省{(1-pkl_size/csv_size)*100:.0f}%)")

【保存数据】


CSV格式:
  ✓ train.csv: 217,162 条
  ✓ val.csv: 27,145 条
  ✓ test.csv: 27,145 条

PKL格式:
  ✓ train.pkl: 217,162 条
  ✓ val.pkl: 27,145 条
  ✓ test.pkl: 27,145 条

train: CSV=220.1MB, PKL=36.4MB (节省83%)

val: CSV=27.8MB, PKL=6.7MB (节省76%)

test: CSV=27.4MB, PKL=6.7MB (节省76%)


In [29]:
print("=" * 60)
print("【完成】")
print("=" * 60)
print(f"""
数据集划分 (8:1:1):
  训练集: {len(train_df):,} 条 (80.0%)
  验证集: {len(val_df):,} 条 (10.0%)
  测试集: {len(test_df):,} 条 (10.0%)

保存位置: D:/010_CodePrograms/L/LLM_su7/
  CSV: train.csv, val.csv, test.csv
  PKL: train.pkl, val.pkl, test.pkl

特征列 (15列):
  文案类: 评论文案, 微博文案, 根评论文案, 父评论文案
  时间: 发布时间
  用户特征: 用户总转发数, 用户总评论数, 用户总点赞数, 用户是否认证
  评论特征: 是否一级评论, 子评论数, 点赞数
  微博特征: 微博转发数, 微博评论数, 微博点赞数 (新增)

文件大小对比:
  CSV总计: {220.1+27.8+27.4:.1f}MB
  PKL总计: {36.4+6.7+6.7:.1f}MB (节省约80%)
""")

【完成】

数据集划分 (8:1:1):
  训练集: 217,162 条 (80.0%)
  验证集: 27,145 条 (10.0%)
  测试集: 27,145 条 (10.0%)

保存位置: D:/010_CodePrograms/L/LLM_su7/
  CSV: train.csv, val.csv, test.csv
  PKL: train.pkl, val.pkl, test.pkl

特征列 (15列):
  文案类: 评论文案, 微博文案, 根评论文案, 父评论文案
  时间: 发布时间
  用户特征: 用户总转发数, 用户总评论数, 用户总点赞数, 用户是否认证
  评论特征: 是否一级评论, 子评论数, 点赞数
  微博特征: 微博转发数, 微博评论数, 微博点赞数 (新增)

文件大小对比:
  CSV总计: 275.3MB
  PKL总计: 49.8MB (节省约80%)

